# CNN_AE Test Implementation

This was the first model we were able to successfully train on and save from the remote GPU server

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/ML final project/code')

# Import the AutoEncoder class
from csc461_cnn_ae import Autoencoder

import torch
from torchvision import transforms
from PIL import Image
import numpy as np
import cv2

# Load the autoencoder model
model = Autoencoder(64)
model.load_state_dict(torch.load('/content/drive/MyDrive/ML final project/Trained_models/trained_model'))
model.eval()


# Load the input eye image
eye_image = Image.open('/content/drive/MyDrive/ML final project/datasets/SingleEyeTest/eye2.jpg')  # Replace with image path

# Define preprocessing transforms
preprocess = transforms.Compose([
    transforms.Resize((128, 128)),  # Match model's input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Match training normalization
])

# Preprocess the image
input_tensor = preprocess(eye_image).unsqueeze(0)  # Add batch dimension

# Get the reconstruction
with torch.no_grad():
    reconstructed_image = model.encoder(input_tensor)


# Convert reconstructed image to numpy
reconstructed_image_np = reconstructed_image.squeeze(0).permute(1, 2, 0).numpy()
reconstructed_image_np = ((reconstructed_image_np + 1) * 127.5).astype(np.uint8)  # Denormalize

# Convert to BGR for OpenCV
reconstructed_image_bgr = cv2.cvtColor(reconstructed_image_np, cv2.COLOR_RGB2BGR)

# Create an iris mask (assume circular iris in the center)
height, width, _ = reconstructed_image_bgr.shape
center = (width // 2, height // 2)
radius = min(height, width) // 4  # Adjust based on images

mask = np.zeros((height, width), dtype=np.uint8)
cv2.circle(mask, center, radius, (255), thickness=-1)  # Create circular mask

# Blend blue hue with the iris
blue_color = np.array([255, 0, 0], dtype=np.uint8)  # Blue in BGR for OpenCV
blue_layer = np.zeros_like(reconstructed_image_bgr)
blue_layer[:] = blue_color

# Apply the mask to the blue layer
iris_hue = cv2.bitwise_and(blue_layer, blue_layer, mask=mask)
reconstructed_with_hue = cv2.addWeighted(reconstructed_image_bgr, 1.0, iris_hue, 0.5, 0)

# Convert back to RGB and save
final_image = cv2.cvtColor(reconstructed_with_hue, cv2.COLOR_BGR2RGB)
final_pil_image = Image.fromarray(final_image)
final_pil_image.save("/content/drive/MyDrive/ML final project/TestResults/output_eye_with_blue_hue_2.jpg")
final_pil_image.show()



<ipython-input-1-02412dc89f7c>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/ML final project/Trained_models/tra

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/ML final project/code')

# Import the Autoencoder class
from csc461_cnn_ae import Autoencoder

import torch
from torchvision import transforms
from PIL import Image
import numpy as np
import cv2

# Precomputed color embeddings (replace with actual precomputed embeddings)
color_embeddings = {
    "blue": torch.load('/content/drive/MyDrive/ML final project/ColorEmbeddings/blue.pt'),
    "brown": torch.load('/content/drive/MyDrive/ML final project/ColorEmbeddings/brown.pt'),
    "green": torch.load('/content/drive/MyDrive/ML final project/ColorEmbeddings/green.pt'),
    "gray": torch.load('/content/drive/MyDrive/ML final project/ColorEmbeddings/gray.pt'),
    "hazel": torch.load('/content/drive/MyDrive/ML final project/ColorEmbeddings/hazel.pt')
}

# Load the autoencoder model
model = Autoencoder(64)
model.load_state_dict(torch.load('/content/drive/MyDrive/ML final project/Trained_models/trained_model_1.1'))
model.eval()

# Define preprocessing transforms
preprocess = transforms.Compose([
    transforms.Resize((128, 128)),  # Match your model's input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Match your training normalization
])

def change_eye_color(eye_image_path, desired_color):
    """
    Change the eye color of the input image to the desired color.

    Args:
        eye_image_path (str): Path to the input eye image.
        desired_color (str): Desired eye color ("blue", "brown", "green", "gray", "hazel").

    Returns:
        PIL.Image: Image with the modified eye color.
    """
    # Load and preprocess the input eye image
    eye_image = Image.open(eye_image_path)
    input_tensor = preprocess(eye_image).unsqueeze(0)  # Add batch dimension

    # Get the embedding for the input image
    with torch.no_grad():
        input_embedding = model.encoder(input_tensor)

    # Retrieve the embedding for the desired color
    if desired_color not in color_embeddings:
        raise ValueError(f"Invalid color: {desired_color}. Choose from {list(color_embeddings.keys())}.")
    desired_color_embedding = color_embeddings[desired_color]

    # Combine the embeddings (simple addition; modify if needed for your use case)
    modified_embedding = input_embedding + desired_color_embedding

    # Decode the modified embedding to get the output image
    with torch.no_grad():
        output_tensor = model.decoder(modified_embedding)

    # Convert the output tensor to an image
    output_image_np = output_tensor.squeeze(0).permute(1, 2, 0).numpy()
    output_image_np = ((output_image_np + 1) * 127.5).astype(np.uint8)  # Denormalize

    # Convert to RGB format
    output_image_rgb = cv2.cvtColor(output_image_np, cv2.COLOR_BGR2RGB)
    final_pil_image = Image.fromarray(output_image_rgb)

    return final_pil_image

# Example usage
output_image = change_eye_color('/content/drive/MyDrive/ML final project/datasets/SingleEyeTest/eye2.jpg', 'blue')
output_image.save('/content/drive/MyDrive/ML final project/TestResults/output_eye_with_blue.jpg')
output_image.show()


<ipython-input-3-f5f1f409643f>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  "blue": torch.load('/content/drive/MyDrive/ML final project/ColorEmbeddings/blue.pt'),
<ipyt

In [ ]:

import sys
sys.path.append('/content/drive/MyDrive/ML final project/code')

import os
import torch
from torchvision import transforms
from PIL import Image
from csc461_cnn_ae import Autoencoder

# Define preprocessing transforms
preprocess = transforms.Compose([
    transforms.Resize((128, 128)),  # Match your model's input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Match your training normalization
])

# Load the trained autoencoder model
model = Autoencoder(64)
model.load_state_dict(torch.load('/content/drive/MyDrive/ML final project/Trained_models/trained_model_1.1'))
model.eval()

def get_embedding_from_reconstructed_image(image_path, model):
    """
    Derive the embedding from a reconstructed image using the model's encoder.

    Args:
        image_path (str): Path to the reconstructed image.
        model: Trained autoencoder model.

    Returns:
        torch.Tensor: Embedding for the reconstructed image.
    """
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(image).unsqueeze(0)  # Add batch dimension

    # Generate embedding
    with torch.no_grad():
        embedding = model.encoder(input_tensor)
    return embedding.squeeze(0)  # Remove batch dimension

# Define the paths for each reconstructed image
reconstructed_images = {
    "blue": "/content/drive/MyDrive/ML final project/reconstructed_imgs/blue.jpg",
    "brown": "/content/drive/MyDrive/ML final project/reconstructed_imgs/brown.jpg",
    "green": "/content/drive/MyDrive/ML final project/reconstructed_imgs/green.jpg",
    "gray": "/content/drive/MyDrive/ML final project/reconstructed_imgs/gray.jpg",
    "hazel": "/content/drive/MyDrive/ML final project/reconstructed_imgs/hazel.jpg"
}

# Compute and save embeddings
color_embeddings = {}
for color, image_path in reconstructed_images.items():
    try:
        print(f"Processing {color} reconstructed image...")
        embedding = get_embedding_from_reconstructed_image(image_path, model)
        color_embeddings[color] = embedding
        torch.save(embedding, f"/content/drive/MyDrive/ML final project/ColorEmbeddings/{color}.pt")
        print(f"Saved embedding for {color}.")
    except Exception as e:
        print(f"Error processing {color} image: {e}")


<ipython-input-2-2ce74c7dec56>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/ML final project/Trained_models/tra

Processing blue reconstructed image...
Saved embedding for blue.
Processing brown reconstructed image...
Saved embedding for brown.
Processing green reconstructed image...
Saved embedding for green.
Processing gray reconstructed image...
Saved embedding for gray.
Processing hazel reconstructed image...
Saved embedding for hazel.
